In [9]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import glob as glob
import random

In [10]:
def gsmooth(x_array, y_array, var_y, vexp = .001, nsig = 5.0):
    #returns smoothed flux array
    # if no variance array create a constant one
    if len(var_y) == 0:
        var_y = np.zeros(len(y_array))
        
    for i in range(len(var_y)):
        if var_y[i] == 0:
            var_y[i] = 1E-20
            # var_y[i] = 1E-31
    
    # Output y-array
    new_y = np.zeros(len(x_array), float)
    
    # Loop over y-array elements
    for i in range(len(x_array)):
        
        # Construct a Gaussian of sigma = vexp*x_array[i]
        gaussian = np.zeros(len(x_array), float)
        sigma = vexp*x_array[i]
        
        # Restrict range to +/- nsig sigma
        sigrange = np.nonzero(abs(x_array-x_array[i]) <= nsig*sigma)
        gaussian[sigrange] = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_array[sigrange]-x_array[i])/sigma)**2)
        
        # Multiply Gaussian by 1 / variance
        W_lambda = gaussian / var_y
        
        # Perform a weighted sum to give smoothed y value at x_array[i]
        W0 = np.sum(W_lambda)
        W1 = np.sum(W_lambda*y_array)
        new_y[i] = W1/W0

    # Return smoothed y-array
    return new_y

In [11]:
def deredshift(wavelength, z):
    dered_wave = wavelength / (1+z)
    return dered_wave

In [12]:
def find_min_wave(dered_wave, sm_flux, w1 = 5900., w2 = 6500.):
    elem_range = np.where((dered_wave > w1) & (dered_wave < w2)) #set domain as within minwave and maxwave
    list(elem_range)
    elem_wavelist = dered_wave[elem_range] 
    elem_fluxlist = sm_flux[elem_range] #find corresponding flux vales for wavelengths within domain
    elem_flux = min(elem_fluxlist) #find minimum value within these flux vales to locate "dip"
    elem_wavelength = dered_wave[np.where(sm_flux == elem_flux)][0] #find the corresponding wavelength
    return(elem_wavelength)

In [13]:
def find_velocity(min_wave, rest_wave = 6355.):
    wavelength_prop = min_wave / rest_wave
    ejecta_velocity = c * (wavelength_prop ** 2 - 1) / (wavelength_prop ** 2 + 1)
    return ejecta_velocity

In [14]:
def get_SN_name(file):
    SN_name = file.split('-')[0]
    return SN_name

In [15]:
"""def make_z_dict(file):
    #reads in CfA4_info.txt
    sn_datatext = open(file, "r")
    sn_datainfo = sn_datatext.readlines()
    z_dict = {}
    start = False
    for line in sn_datainfo:
        if line.split()[0] == '2006ct':
                start = True
        if start:
            line_elems = line.split()
            name = line_elems[0]
            z = line_elems[10]
            z_dict[name] = float(z)
    return z_dict"""

'def make_z_dict(file):\n    #reads in CfA4_info.txt\n    sn_datatext = open(file, "r")\n    sn_datainfo = sn_datatext.readlines()\n    z_dict = {}\n    start = False\n    for line in sn_datainfo:\n        if line.split()[0] == \'2006ct\':\n                start = True\n        if start:\n            line_elems = line.split()\n            name = line_elems[0]\n            z = line_elems[10]\n            z_dict[name] = float(z)\n    return z_dict'

In [16]:
def make_z_dict(file):
    #reads in CfA4_info.txt
    sn_datatext = open(file, "r")
    sn_datainfo = sn_datatext.readlines()
    z_dict = {}
    start_line = 60
    for line in sn_datainfo[start_line:]:
        line_elems = line.split()
        temp = line_elems[0]
        if temp[0] == "2":
            name = "sn" + temp
        else:
            name = temp
        z = line_elems[10]
        z_dict[name] = float(z)
    return z_dict

In [17]:
def vel_error_dist(wavelength, flux, flux_error, n = 1000):
    velocity_distribution = []
    minwave = 6000
    maxwave = 7000
    wavelength_rest = 6355
    for i in range(n):
        vexp= np.random.normal(.0015,.00025) 
        smooth_flux = gsmooth(wavelength, flux, flux_error,vexp)
        SiII_obs = find_wave_obs(wavelength, smooth_flux, minwave, maxwave)
        v = find_velocity(SiII_obs, wavelength_rest)
        #print (v, vexp, SiII_obs) this line can be uncommented to analyze increments of the loop individually
        velocity_distribution.append(v)
    vmean = np.mean(velocity_distribution)
    vstd = np.std(velocity_distribution)
    plt.hist(velocity_distribution)
    return "mean velocity is", vmean, "km/s", "standard deviation is", vstd, "km/s"

In [18]:
#Parse ned for associated galaxy
#use astroquery to find z value
def find_z(SN_name):
    query_url = 'https://ned.ipac.caltech.edu/cgi-bin/objsearch?objname=sn2007fs&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES'
    sn_url = query_url.replace("sn2007fs", SN_name)
    return sn_url

In [19]:
def plotspectra(wavelength, flux, dered_wave, sm_flux):
    plt.figure(figsize=(20, 10))
    plt.axvline(x=6355, linewidth=.5)
    #resting wavelength of Si II
    plt.plot(wavelength, flux, linewidth=1, color='red') #redshifted
    plt.plot(dered_wave, sm_flux, linewidth=1, color='blue') #deredshifted
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Relative Flux')

In [20]:
#TEST YOUR CODE ON ONE OBJECT (plot original spectrum, deredshifted spectrum, and the smoothed deredshifted spectrum, print velocity)
#YOUR CODE GOES HERE
my_Supernova = ("sn2007hj-20070903.28-fast.flm")

wavelength, flux, flux_error = np.loadtxt(my_Supernova, unpack = True)
z = 0.003
dered_wave = deredshift(wavelength, z)
sm_flux = gsmooth(dered_wave, flux, flux_error)

plotspectra(wavelength, flux, dered_wave, sm_flux)


FileNotFoundError: [Errno 2] No such file or directory: 'sn2007hj-20070903.28-fast.flm'

In [21]:
#YOUR CODE GOES HERE
z_dict = make_z_dict("CfA4_info.txt")
sndata = glob.glob("*.flm")
wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
z = 0.017192 #obtain from NED
c = 299792 #speed of light in km/s
print(sndata)
dered_wave = deredshift(wavelength, z)
sm_flux = gsmooth(dered_wave, flux, flux_error)
min_wave = find_min_wave(dered_wave, sm_flux)
print(min_wave)
v = find_velocity(min_wave)
print(v)

['sn2007fsdata.flm', 'sn2007hjdata.flm', 'sn2007fbdata.flm', 'sn2007Adata.flm', 'sn2007jgdata.flm']
6129.28532666
-10836.8784392


In [22]:
make_z_dict("CfA4_info.txt")

{'PTF10bjs': 0.03,
 'sn2006ct': 0.0315,
 'sn2006ou': 0.0135,
 'sn2007A': 0.0177,
 'sn2007aj': 0.011,
 'sn2007bj': 0.0167,
 'sn2007cb': 0.0366,
 'sn2007cc': 0.0291,
 'sn2007cf': 0.0329,
 'sn2007cn': 0.0253,
 'sn2007cs': 0.0176,
 'sn2007ev': 0.0427,
 'sn2007fb': 0.018,
 'sn2007fq': 0.0425,
 'sn2007fs': 0.0172,
 'sn2007hg': 0.03,
 'sn2007hj': 0.0141,
 'sn2007hu': 0.0354,
 'sn2007if': 0.0742,
 'sn2007ir': 0.0352,
 'sn2007is': 0.0297,
 'sn2007jg': 0.0371,
 'sn2007kd': 0.0242,
 'sn2007kf': 0.046,
 'sn2007kg': 0.007,
 'sn2007kh': 0.05,
 'sn2007kk': 0.041,
 'sn2007le': 0.0067,
 'sn2007nq': 0.045,
 'sn2007ob': 0.0339,
 'sn2007rx': 0.0301,
 'sn2007ss': 0.0155,
 'sn2007su': 0.0279,
 'sn2007sw': 0.0252,
 'sn2007ux': 0.0309,
 'sn2008A': 0.0165,
 'sn2008C': 0.0166,
 'sn2008Q': 0.06,
 'sn2008Y': 0.0697,
 'sn2008Z': 0.021,
 'sn2008ac': 0.0528,
 'sn2008ad': 0.05,
 'sn2008ae': 0.0301,
 'sn2008ai': 0.0353,
 'sn2008ar': 0.0261,
 'sn2008at': 0.035,
 'sn2008bi': 0.0134,
 'sn2008bw': 0.0331,
 'sn2008by': 0.0

In [35]:
class Supernova:
    def __init__(file):
        file.name = get_SN_name(file)
        file.data = wavelength, flux, flux_error = np.loadtxt(file[0], unpack = True)

In [46]:
#FINAL SCRIPT STEPS
#1) Get array of all flm file names
#2) Make redshift dictionary
#3) loop through files
    #a) read in spectrum
    #b) get SN name from file name
    #c) get redshift from dictionary
    #d) deredshift
    #e) smooth
    #f) find wavelength where Si line is minimum
    #g) find and print velocity (and SN name)

In [49]:
sndata = glob.glob("*.flm")

z_dict = make_z_dict("CfA4_info.txt")
for i in range(len(sndata)):
    name = get_SN_name(sndata[i])
    wavelength, flux, flux_error = np.loadtxt(sndata[i], unpack = True)
    dered_wave = deredshift(wavelength, z_dict[name])
    sm_flux = gsmooth(dered_wave, flux, flux_error)
    min_wave = find_min_wave(dered_wave, sm_flux)
    v = find_velocity(min_wave)
    print (v)

-12480.1227903
-10611.0637019
-11752.8635862
-10839.2331491
-12759.6282317
